# MOUTH CROPPING

This tutorial shows how to use our `AVSRDataLoader` to extract cropped mouth.

**Note** This tutorial requires `mediapipe` or `retinaface` detector. Please refer to [preparation](../preparation#setup) for installation.

**Note** To run this tutorial, please make sure you are in tutorials folder.

In [ ]:
import sys
sys.path.insert(0, "../")

In [ ]:
import os
import torch
import torchvision

from preparation.data.data_module import AVSRDataLoader

## 1. Initialize AVSR Dataloader.
    
The AVSRDataLoader class is a pre-process pipeline from raw video to cropped mouth, which includes face detection, landmarks tracking, face transformation and mouth cropping. We provide `mediapipe` and `retinaface` face detector, `mediapipe` is faster with relatively lower precision compared with `retinaface`.

In [ ]:
class AVSRDataLoader(torch.nn.Module):
    def __init__(self, detector="retinaface"):
        super().__init__()
        if detector == "mediapipe":
            from preparation.detectors.mediapipe.detector import LandmarksDetector
            from preparation.detectors.mediapipe.video_process import VideoProcess
            self.landmarks_detector = LandmarksDetector()
            self.video_process = VideoProcess(convert_gray=False)
        elif detector == "retinaface":
            from preparation.detectors.retinaface.detector import LandmarksDetector
            from preparation.detectors.retinaface.video_process import VideoProcess
            self.landmarks_detector = LandmarksDetector(device="cuda:0")
            self.video_process = VideoProcess(convert_gray=False)

    def forward(self, data_filename):
        video = self.load_video(data_filename)
        landmarks = self.landmarks_detector(video)
        video = self.video_process(video, landmarks)
        video = torch.tensor(video)
        return video

    def load_video(self, data_filename):
        return torchvision.io.read_video(data_filename, pts_unit="sec")[0].numpy()

In [ ]:
video_dataloader = AVSRDataLoader(detector="retinaface")

## 2. Download a video from web

In [ ]:
!wget --content-disposition http://www.doc.ic.ac.uk/~pm4115/autoAVSR/autoavsr_demo_video.mp4 -O ./input.mp4
data_filename = "input.mp4"

In [ ]:
preprocessed_video = video_dataloader(data_filename)

## 3. Saved the cropped video

In [ ]:
def save2vid(filename, vid, frames_per_second):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    torchvision.io.write_video(filename, vid, frames_per_second)
save2vid("./output.mp4", preprocessed_video, frames_per_second=30)